In [ ]:
# kiểm tra xem có ảnh nào không đọc được không 


In [1]:
# test chức năng resize
import cv2

In [5]:
img = cv2.imread(r"D:\code\projects\mathematical_formula_classification\data\processed\processed_image\ky_hieu_tich\nhan_1_2.png")


In [ ]:
! git add .
! git commit -m "test_new"
! git push

git: 'git' is not a git command. See 'git --help'.

The most similar command is
	init
Everything up-to-date


In [6]:

img

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [2]:
import torchvision.transforms as transforms
data_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1), # Chuyển ảnh sang ảnh xám (1 kênh)
    transforms.ToTensor(),                       # Chuyển ảnh PIL thành PyTorch Tensor (giá trị 0-1)
    transforms.Normalize([0.5], [0.5])           # Chuẩn hóa tensor về khoảng [-1, 1]
])
type(data_transforms)

torchvision.transforms.transforms.Compose

In [2]:
import os
import shutil
import random
from glob import glob


In [3]:



SOURCE_DIRECTORY = r'..\data\processed\train'
DESTINATION_DIRECTORY = r'dataset'
GET_MUNBER = 0.4

In [4]:
class_dirs = [ d for d in os.listdir(SOURCE_DIRECTORY) if os.path.isdir(os.path.join(SOURCE_DIRECTORY,d))]
class_dirs

['ky_hieu_tich', 'ky_hieu_tong_can', 'ky_hieu_tong_sigma_images']

In [7]:
for class_folder in class_dirs:
    source_class_path = os.path.join(SOURCE_DIRECTORY , class_folder)
    dest_class_path = os.path.join(DESTINATION_DIRECTORY , class_folder)
    
    # tao thu muc chua 
    os.makedirs(dest_class_path, exist_ok= True)
    
    # lay danh sach file trong class
    lst_file_class = glob(os.path.join(source_class_path, '*'))
    lst_get_random_file_class = random.sample(lst_file_class, int(len(lst_file_class) * GET_MUNBER))
    
    for copy_file in lst_get_random_file_class:
        shutil.copy(copy_file, dest_class_path)


['..\\data\\processed\\train\\ky_hieu_tong_sigma_images\\tong_sigma_0_1.png',
 '..\\data\\processed\\train\\ky_hieu_tong_sigma_images\\tong_sigma_0_1_aug_1.png',
 '..\\data\\processed\\train\\ky_hieu_tong_sigma_images\\tong_sigma_0_1_aug_10.png',
 '..\\data\\processed\\train\\ky_hieu_tong_sigma_images\\tong_sigma_0_1_aug_2.png',
 '..\\data\\processed\\train\\ky_hieu_tong_sigma_images\\tong_sigma_0_1_aug_3.png',
 '..\\data\\processed\\train\\ky_hieu_tong_sigma_images\\tong_sigma_0_1_aug_5.png',
 '..\\data\\processed\\train\\ky_hieu_tong_sigma_images\\tong_sigma_0_1_aug_6.png',
 '..\\data\\processed\\train\\ky_hieu_tong_sigma_images\\tong_sigma_0_1_aug_7.png',
 '..\\data\\processed\\train\\ky_hieu_tong_sigma_images\\tong_sigma_0_1_aug_9.png',
 '..\\data\\processed\\train\\ky_hieu_tong_sigma_images\\tong_sigma_100_1e.png',
 '..\\data\\processed\\train\\ky_hieu_tong_sigma_images\\tong_sigma_100_1e_aug_1.png',
 '..\\data\\processed\\train\\ky_hieu_tong_sigma_images\\tong_sigma_100_1e_aug_3.

In [ ]:
i = 
i

'..\\data\\processed\\train\\ky_hieu_tong_sigma_images\\tong_sigma_0_1_aug_9.png'